# GeoJupyter demo

## Today: [🔗JupyterGIS](https://jupytergis.readthedocs.io/)

JupyterGIS is a **real-time collaborative** Geographical Information System (GIS) environment in JupyterLab.

You can [🔗try it right now in JupyterLite](https://jupytergis.readthedocs.io/en/latest/lite/lab/)!

Let's explore some functionality together (based on [🔗Carl Boettiger](https://ourenvironment.berkeley.edu/people/carl-boettiger)'s [🔗ESPM-288 course](https://espm-288.carlboettiger.info/)). We'll explore whether neighborhoods that were highly-rated (A) under the disciminatory 1930s practice of [🔗redlining](https://en.wikipedia.org/wiki/Redlining) are greener today than neighborhoods graded D?

### Constants

Some variables that will be used throughout the Notebook.

In [ ]:
from pathlib import Path

DATA_DIR = Path().cwd() / "data"
INEQUALITY_GEOJSON_FILE = DATA_DIR / "redlining_newhaven_ct.geojson"
NDVI_FILE = DATA_DIR / "ndvi.tif"

### Get historical redlining data

We're using [🔗DuckDB](https://duckdb.org/) to connect to a [🔗geopackage](https://www.geopackage.org/) dataset containing data about redlining, and filter that data to select residential neighborhoods in New Haven, Connecticut, USA.

In [ ]:
import ibis
from ibis import _


con = ibis.duckdb.connect(extensions=["spatial"])

redlines = (
    con
    .read_geo("/vsicurl/http://dsl.richmond.edu/panorama/redlining/static/mappinginequality.gpkg")
    .filter(_.city == "New Haven", _.residential)
)

new_haven_redlining =  redlines.execute().set_crs("EPSG:4326")
new_haven_redlining.to_file(INEQUALITY_GEOJSON_FILE, engine="fiona")

new_haven_bbox = new_haven_redlining.total_bounds

Let's explore the data a little bit. Hover over the polygons after running the cell below! Does anything jump out at you?

In [ ]:
new_haven_redlining.explore(column="grade", cmap="inferno")

### Calculating NDVI

We're going to calculate NDVI from Sentinel-2 data.

#### Open Sentinel-2 data

We are using a [🔗STAC catalog](https://stacspec.org/en) to locate the data files we're interested in (covering New Haven during Summer 2024, with <20% cloud cover) and opening them as an Xarray DataSet.

In [ ]:
import odc.stac
from pystac_client import Client

items = Client.open(
    "https://earth-search.aws.element84.com/v1"
).search(
    collections = ['sentinel-2-l2a'],
    bbox=new_haven_bbox,
    datetime = "2024-06-01/2024-09-01",
    query={"eo:cloud_cover": {"lt": 20}}
).item_collection()

data = odc.stac.load(
    items,
    bands=["nir08", "red"],
    bbox=new_haven_bbox,
    resolution=10,
    groupby="solar_day",
    chunks = {},  # this tells odc to use dask
)
data

#### Do the NDVI calculation

In [ ]:
ndvi = (
    (data.nir08 - data.red) / (data.red + data.nir08)
).median(
    "time",
    keep_attrs=True,
).where(
    ndvi < 1
).compute()

ndvi.plot.imshow()

#### Save the NDVI raster to file

In [ ]:
import rioxarray

ndvi.rio.reproject(
    "EPSG:4326",
).rio.to_raster(
    raster_path=NDVI_FILE, 
    driver="COG",
)

### Calculating mean NDVI for each New Haven neighborhood

To find out whether neighborhoods graded "A" are greener than neighborhoods graded "D", we'll calculate the mean NDVI for each neighborhood using [🔗exactextract](https://isciences.github.io/exactextract/background.html), which is known for its capability to include fractional grid cells in its calculation (as opposed to other tools, where a cell is binary, either in or out).

In [ ]:
from exactextract import exact_extract

new_haven_redlining_and_ndvi = exact_extract(
    NDVI_FILE,
    new_haven_redlining,
    "mean_ndvi=mean",
    include_geom = True,
    include_cols=["label", "grade", "city", "fill"],
    output="pandas",
)

new_haven_redlining_and_ndvi.explore(column="mean_ndvi")

## Future

### Story maps / "scrolly telling"

Story map support for JupyterGIS is in progress.

We anticipate working with the [🔗MyST](https://mystmd.org/) and [🔗Closeread](https://closeread.dev/) developers to develop interactive scrollytelling experiences in MyST Markdown documents.

### "microgis" (placeholder name)

We're working on a [🔗project](https://github.com/geojupyter/jupyter-microgis) to provide an instant layered visual environment for any number of Python datasets (starting with rioxarray DataArrays and GeoPandas GeoDataFrames) in a widget.
The goal is to minimize time-to-visualization.

It would provide sensible default symbology choices, and customization would be available with as-needed complexity.
In other words, you shouldn't need to learn a complex symbology expression language when your needs are simple, but complex expression is available if you need it.

```python
from microgis import explore


explore(
    da1, da2, gdf1,
    {
        "data": gdf2,
        "symbology": {
            "choropleth": {
                "steps": 11,
                "classification": "natural",
            },
        },
    },
)
```

### More!

:::{image} https://geojupyter.org/assets/images/community-diagram.svg
:width: 400px
:align: center
:::

GeoJupyter's priorities are broad, and are based on our community's needs. We can only know what those needs are if you join us!

Please join the [🔗Jupyter Zulip](https://jupyter.zulipchat.com) today and find us in the `#geojupyter` channel!

